In [115]:
import pandas as pd
import re
import ast
import gunshot_utils as utils
import importlib
import os
import random
import pandas as pd
import torchaudio

importlib.reload(utils)

<module 'gunshot_utils' from '/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/gunshot_utils.py'>

In [116]:
audio_csv = pd.read_csv('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Audio/Gunshots/csv/edge-collected-gunshot-audio/updated_gunshot_metadata.csv')

In [117]:
audio_csv = audio_csv[['filename', 'firearm', 'caliber', 'gunshot_location_in_seconds', 'num_gunshots']]

In [118]:
audio_csv

,filename,firearm,caliber,gunshot_location_in_seconds,num_gunshots
0,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.72269841],1
1,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.67290249],1
2,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.61977324 3.50795918 5.42746032],3
3,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.75 1.98768707 2.26022676],3
4,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.75 2.61845805 3.06664399],3
...,...,...,...,...,...
2143,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.7406576],1
2144,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.66732426],1
2145,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.53469388 1.74122449 2.14659864 2.3253288 2...,5
2146,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.75 1.94283447 2.34698413 2.52979592 2...,5


In [119]:
LOUDNESS_THRESHOLD = -10

filtered_df = utils.filter_loud_gunshots(audio_csv, filepath_column='filename', loudness_threshold=LOUDNESS_THRESHOLD)

-11.202961
-9.96608
-9.906976
-11.508646
-10.234955
-12.086302
-10.160595
-11.852609
-11.399101
-10.730372
-11.437885
-11.457085
-10.277843
-10.341444
-11.148655
-10.172982
-10.096908
-10.534211
-10.241501
-10.58507
-9.360193
-11.507725
-11.277571
-11.309326
-11.056451
-11.41467
-11.31117
-11.588105
-9.751564
-10.932682
-10.726435
-9.872534
-9.926373
-9.674832
-9.62947
-9.733822
-9.072835
-9.12414
-9.507083
-9.189647
-9.14336
-15.108186
-15.984938
-14.62915
-9.169128
-15.488562
-15.513438
-16.388758
-15.393342
-13.536947
-16.0433
-14.640311
-15.339089
-9.131316
-11.864495
-11.01406
-12.44231
-12.261759
-11.5924225
-11.984161
-11.118589
-11.199294
-12.246408
-10.9463215
-11.788977
-11.401052
-11.5312395
-11.190213
-11.826754
-11.379654
-13.609694
-15.560212
-15.6215515
-15.720192
-16.57132
-13.835197
-16.62919
-11.868061
-11.89548
-15.13237
-15.204773
-13.797894
-10.066103
-10.003404
-11.125416
-11.479402
-11.048803
-11.991844
-11.866288
-11.186724
-12.266355
-12.251108
-11.808842
-12.5

/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/gunshot_utils.py:650: UserWarning: PySoundFile failed. Trying audioread instead.
  def filter_by_firearm(df, gun_types):
/Users/borosabel/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


-41.830746
-41.70525
-41.85219
-41.91361
-40.967083
-40.749683
-42.428707
-42.190876
-42.513214
-42.454067
-42.38463
-42.22195
-43.525322
-43.37052
-45.686462
-45.47528
-32.081387
-31.624575
-35.873848
-36.234447
-31.418818
-30.83701
-32.71151
-32.02623
-32.853203
-32.32522
-32.610138
-32.152863
-33.90302
-33.32685
-35.561333
-34.95772
-32.907578
-35.814976
-32.17777
-33.55574
-33.53795
-33.362305
-34.647717
-36.477142
-31.893494
-35.20538
-31.031616
-32.21847
-32.518944
-32.34205
-33.04281
-35.184017
-39.99762
-40.03132
-41.984154
-40.894276
-39.01608
-39.137554
-41.18179
-41.21973
-40.500988
-40.851864
-40.327293
-40.53099
-42.114
-41.46034
-43.829514
-43.893322
-40.48225
-41.186447
-39.403126
-41.216522
-41.035927
-40.82911
-41.39029
-44.14639
-21.746023
-27.281815
-23.425564
-19.253382
-23.09031
-18.38481
-18.3773
-23.503963
-19.486578
-21.70923
-27.97026
-25.091322
-23.279507
-27.477306
-25.532978
-21.858442
-27.592403
-23.960056
-24.1173
-26.05876
-25.02104
-23.273571
-29.53064
-

In [120]:
filtered_df

,filename,firearm,caliber,gunshot_location_in_seconds,num_gunshots,Max Loudness (dB)
1,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.67290249],1,-9.966080
2,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.61977324 3.50795918 5.42746032],3,-9.906976
20,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Remington 870,12 guage,[1.59827664],1,-9.360193
28,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.77839002],1,-9.751564
31,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.62344671],1,-9.872534
...,...,...,...,...,...,...
2143,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.7406576],1,-2.090497
2144,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.66732426],1,-1.903236
2145,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.53469388 1.74122449 2.14659864 2.3253288 2...,5,-0.692698
2146,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.75 1.94283447 2.34698413 2.52979592 2...,5,-0.715194


In [121]:
filtered_df['gunshot_location_in_seconds'] = filtered_df['gunshot_location_in_seconds'].apply(
    lambda x: utils.preprocess_gunshot_times(x)
)

In [122]:
glock_df = utils.filter_by_firearm(filtered_df, ['glock'])

In [123]:
glock_df.to_pickle('./pkl_data/gunshot_data_glock_only_5db.pkl')

In [ ]:
# Let's also resample the whole music dataset and convert to .wav - only need to do it once.

In [51]:
music_df = pd.read_excel('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Excel/baseline_data_w_topics_w_features.xlsx', engine='openpyxl')

In [65]:
resampled_df = utils.resample_and_convert_to_wav(music_df, 'Path', './Data/music', limit=300)

Processing audio files: 100%|██████████| 300/300 [01:46<00:00,  2.82it/s]
/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/gunshot_utils.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df.dropna(subset=['Path'])
/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/gunshot_utils.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['file_extension'] = '.wav'


In [66]:
resampled_df.to_pickle('./pkl_data/music_data_resampled_300.pkl')